In [8]:
from datasets import load_dataset

from transformers import (
    LongformerTokenizer,
    LongformerForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

In [2]:
dataset = load_dataset("owaiskha9654/PubMed_MultiLabel_Text_Classification_Dataset_MeSH")

README.md:   0%|          | 0.00/960 [00:00<?, ?B/s]

(…)ext Classification Dataset Processed.csv:   0%|          | 0.00/120M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## First we will pretrain our longformer for masked word on textual data, so weights are updated for biomedical language


In [19]:
list(dataset['train'].features.keys())

['Title',
 'abstractText',
 'meshMajor',
 'pmid',
 'meshid',
 'meshroot',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'L',
 'M',
 'N',
 'Z']

In [9]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
model = LongformerForMaskedLM.from_pretrained("allenai/longformer-base-4096")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

In [24]:
def tokenize_function(examples):
    return tokenizer(
        examples["abstractText"],
        padding="max_length",
        truncation=True,
        max_length=4096,  
    )

In [25]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['Title','abstractText','meshMajor','pmid','meshid',
 'meshroot', 'A' 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M','N','Z'])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [28]:
print(tokenized_datasets['train'][0])

{'input_ids': [0, 597, 22129, 12, 10231, 2242, 3707, 179, 14224, 11576, 9042, 31, 1484, 19, 25599, 2911, 19036, 36, 2146, 1200, 43, 8, 19, 29851, 1668, 36, 3103, 1200, 43, 58, 24305, 4, 33264, 21, 12333, 8, 2006, 11, 80, 5612, 4, 30800, 6, 634, 4281, 9770, 6, 4986, 41217, 5708, 26929, 58, 32755, 131, 36013, 5, 1468, 4634, 4756, 21, 13773, 30, 9284, 1938, 5448, 634, 25655, 261, 40143, 1242, 219, 2794, 1105, 12, 510, 22434, 4513, 4, 13064, 13436, 9, 1907, 231, 6, 365, 6, 545, 6, 504, 6, 2357, 58, 2006, 4, 20, 181, 12, 4540, 8151, 21, 8446, 12198, 30, 13998, 2678, 661, 43941, 5448, 4, 33264, 7910, 21, 747, 3059, 19, 25599, 2911, 19036, 8, 29851, 1668, 4, 96, 29851, 1668, 4412, 33264, 545, 8, 504, 19, 239, 15, 438, 23982, 801, 58, 303, 4, 20, 181, 12, 4540, 21, 1455, 7154, 6, 8, 11, 2289, 20631, 4, 440, 22792, 21, 6373, 227, 2621, 9, 181, 12, 4540, 8, 13064, 13436, 5708, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [29]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True, 
    mlm_probability=0.15,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./longformer-mlm-pretrained",  # Directory to save the model
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=2,  # Smaller batch size due to larger sequences
    save_steps=5000,  # Save checkpoint every 500 steps
    save_total_limit=2,  # Keep only the last two checkpoints
    logging_dir="./logs",  # Directory for logs
    logging_steps=1000,  # Log metrics every 100 steps
    prediction_loss_only=True, 
    learning_rate=5e-5,  # Learning rate
    report_to="none",  # Disable default reporting
    fp16=True, 
)
